# ***DISTRIBUCIÓN DE SEGMENTOS***

### DataFrames:

In [2]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
if nombre.startswith("df_")  and "puebla" in nombre and 'df_alfa_q_puebla' not in nombre and 'financiamientos' not in nombre and 'grupos' not in nombre and 'salarios' not in nombre and 'publicacion' not in nombre and 'df_feb_2024_puebla' not in nombre ]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_agosto_2024_puebla',
 'df_alfa_febrero_2024_puebla',
 'df_alfa_julio_2024_puebla',
 'df_alfa_junio_2024_puebla',
 'df_alfa_septiembre_2024_puebla',
 'df_jul_2023_puebla',
 'df_mar_2024_puebla',
 'df_may_2024_puebla',
 'df_sep_2023_puebla']

In [3]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    # Verificar si la columna 'm2_contruido' existe en el DataFrame
    #if 'm2_contruido' in df.columns:
        # Renombrar la columna
    df.rename(columns={'segmento':'categoria','Categoria':'categoria','Category':'categoria','Precio':'precio'}, inplace=True)
    # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [4]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id', 'categoria']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

KeyError: "['id'] not in index"

### CONTEO ÚNICO DE CATEGORÍAS

In [5]:
conteo_total = {}
for df in dataframes_list:
    conteo_actual = df['categoria'].value_counts().to_dict()
    # Agregar el conteo actual al conteo total
    for categoria, conteo in conteo_actual.items():
        conteo_total[categoria] = conteo_total.get(categoria, 0) + conteo

conteo_total_df = pd.DataFrame(list(conteo_total.items()), columns=['categoria', 'conteo_total'])
conteo_total_df

,categoria,conteo_total
0,E2,2372
1,S1,2361
2,E3,1525
3,,1543
4,D2,924
5,S3,784
6,D1,775
7,C1,838
8,S2,787
9,A3,734


In [6]:
# Añadir una nuevas columnas que extraigan el número y letra de la categoría
conteo_total_df['segmento'] = conteo_total_df['categoria'].str[0]
conteo_total_df['clase'] = conteo_total_df['categoria'].str.extract(r'(\d+)').fillna('1')
conteo_total_df.drop(columns=['categoria'], inplace=True)

nueva_fila_df = pd.DataFrame({'conteo_total': [0], 'segmento': ['E'], 'clase': ['3']})
conteo_total_dfs = pd.concat([conteo_total_df, nueva_fila_df], ignore_index=True)
conteo_total_dfs

,conteo_total,segmento,clase
0,2372,E,2
1,2361,S,1
2,1525,E,3
3,1543,NaN,1
4,924,D,2
5,784,S,3
6,775,D,1
7,838,C,1
8,787,S,2
9,734,A,3


In [7]:
#segmentos_dist = conteo_total_df.groupby(['segmento', 'clase']) # Agrupar
segmentos_dist = conteo_total_dfs.groupby(['segmento', 'clase'])['conteo_total'].sum()
segmentos_dist

segmento  clase
A         1         543
          2         496
          3         734
B         1         536
          2         835
          3         527
C         1         838
          2         716
          3         924
D         1         775
          2         924
          3         785
E         1         823
          2        2372
          3        1525
L         1         441
          2         182
          3         101
S         1        2361
          2         787
          3         784
Name: conteo_total, dtype: int64

## Porcentajes totales de registros por segmento

In [8]:
# Calcular el total de registros por cada segmento, incluyendo todas sus categorías
total_por_segmento = conteo_total_dfs.groupby('segmento')['conteo_total'].sum() 
# Calcular el total de registros en todos los segmentos
total_registros_total = total_por_segmento.sum()
# Calcular el porcentaje que representa cada total de segmento con respecto al total de registros en todo el DataFrame
porcentaje_total_segmento = (total_por_segmento / total_registros_total) * 100
# DataFrame con los resultados
resultados = pd.DataFrame({
    'segmento': total_por_segmento.index,
    'total_por_segmento': total_por_segmento.values,
    'porcentaje_del_total': porcentaje_total_segmento.values
})

print("Total de registros: ",total_registros_total)
#porcentaje_del_total_porsegmentos
resultados

Total de registros:  18009


,segmento,total_por_segmento,porcentaje_del_total
0,A,1773,9.845077
1,B,1898,10.539175
2,C,2478,13.759787
3,D,2484,13.793103
4,E,4720,26.209118
5,L,724,4.020212
6,S,3932,21.833528


In [9]:
#Obtener el total de registros por segmento y clase
total_segmento = conteo_total_df.groupby('segmento')['conteo_total'].sum()
#Calcular los porcentajes
segmentos_percent = conteo_total_df.merge(total_segmento, on='segmento', suffixes=('', '_total'))
segmentos_percent['porcentaje'] = segmentos_percent['conteo_total'] / segmentos_percent['conteo_total_total'] * 100
#Seleccionar las columnas necesarias y eliminar las columnas auxiliares
segmentos_percent = segmentos_percent[['segmento', 'clase', 'porcentaje']]
segmentos_percent

,segmento,clase,porcentaje
0,E,2,50.254237
1,E,3,32.309322
2,E,1,12.478814
3,E,1,4.957627
4,S,1,60.045778
5,S,3,19.938962
6,S,2,20.015259
7,D,2,37.198068
8,D,1,31.199678
9,D,3,31.602254


Porcentaje de cada clase en relación al % que representa su segmento del total.

In [10]:
temp_dfs = [] # Almacenar los DataFrames temporales de cada segmento
# Realizamos un bucle sobre cada segmento
for segmento in resultados['segmento'].unique():
    # Filtramos las clases correspondientes al segmento actual
    clases_segmento = segmentos_percent[segmentos_percent['segmento'] == segmento]
    
    # Calculamos el total de porcentaje del segmento actual
    total_porcentaje_segmento = resultados[resultados['segmento'] == segmento]['porcentaje_del_total'].iloc[0]
    
    # Calculamos el porcentaje que representa cada clase en relación al porcentaje del segmento
    porcentaje_clase = (clases_segmento['porcentaje'] / 100) * total_porcentaje_segmento
    
    # DataFrame temporal para el segmento actual
    temp_df = pd.DataFrame({
        'segmento': [segmento] * len(clases_segmento),
        'clase': clases_segmento['clase'].values,
        'porcentaje_clase': porcentaje_clase#.round()
    })
    
    # Agrega DataFrame temporal a la lista
    temp_dfs.append(temp_df)

# Concatena todos los DataFrames temporales en un solo DataFrame
porcentajes_clase = pd.concat(temp_dfs, ignore_index=True)
porcentajes_clase

,segmento,clase,porcentaje_clase
0,A,3,4.075740
1,A,2,2.754178
2,A,1,3.015159
3,B,2,4.636571
4,B,1,2.976290
5,B,3,2.926315
6,C,1,4.653229
7,C,3,5.130768
8,C,2,3.975790
9,D,2,5.130768


In [11]:
# Agrupar por 'segmento' y 'clase', y sumar los porcentajes
porcentajes_clase_agrupado = porcentajes_clase.groupby(['segmento', 'clase']).agg({'porcentaje_clase': 'sum'}).reset_index()

# Luego aplicar pivot
porcentajes_clase_ordenado = porcentajes_clase_agrupado.pivot(index='segmento', columns='clase', values='porcentaje_clase')
porcentajes_clase_ordenado.fillna(0, inplace=True)
porcentajes_clase_ordenado

clase,1,2,3
segmento,,,
A,3.015159,2.754178,4.075740
B,2.976290,4.636571,2.926315
C,4.653229,3.975790,5.130768
D,4.303404,5.130768,4.358932
E,4.569937,13.171192,8.467988
L,2.448776,1.010606,0.560831
S,13.110112,4.370037,4.353379


## Gráfica

In [12]:
segmentos = list(porcentajes_clase_ordenado.index)
segmentos_sorted = ['E','D','C','B','A','S','L']  # Orden deseado
clases = porcentajes_clase_ordenado.columns # Clases como etiquetas de las series de datos
clases = ['1', '2', '3']  # Clases en el orden deseado
# Asegúrate de que todos los segmentos mencionados estén en los datos con valor 0 si no existen
for segmento in segmentos_sorted:
    if segmento not in porcentajes_clase_ordenado.index:
        porcentajes_clase_ordenado.loc[segmento] = [0] * len(clases)

# Reordenar el DataFrame para seguir el orden deseado en `segmentos_sorted`
porcentajes_clase_ordenado = porcentajes_clase_ordenado.loc[segmentos_sorted]
colores = {
    '1': '#2962ff',
    '2': '#9500ff',
    '3': '#ff0059'
}
# Almacenar las barras de cada clase
barras = []
for clase in clases:
    barras.append(go.Bar(
        y=segmentos_sorted,  # Eje y: Segmentos
        x=porcentajes_clase_ordenado[clase],  # Eje x: Porcentajes de clase
        name=f'{clase}',  # Nombre de la serie de datos
        orientation='h',  # Orientación horizontal
        text=porcentajes_clase_ordenado[clase].apply(lambda x: f'{x:.2f}%'),  # Texto con los porcentajes
        textposition='auto',  # Posición automática del texto
        marker_color=colores[clase]   # Color de las barras
    ))

fig = go.Figure(data=barras)
fig.update_layout(
    yaxis=dict(
        title='Segmento', # Título del eje y        
    ),
    legend_title="Clase",
    barmode='stack',
    legend=dict(
        orientation='h',  # Orientación horizontal de la leyenda
        yanchor='bottom',  # Anclar al borde inferior
        y=1.02,  # Colocar la leyenda justo debajo del gráfico
        xanchor='right',
        x=1
    ),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico      
    xaxis=dict(
        title='Porcentaje', # Título del eje x
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        gridwidth=1,
    ),  
    margin=dict(l=10, r=10, t=10, b=10)  # Ajusta los márgenes (left, right, top, bottom)
)

# Función para guardar la gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
guardar_grafico_como_html(fig, 'g_bar_dist_segmentos', carpeta='assets\graficas')

fig.show()

Actualizar lista de segmentos y orden de clase